In [1]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install streamlit
!pip install pyngrok

     |████████████████████████████████| 8.0 MB 8.6 MB/s 
     |████████████████████████████████| 111 kB 69.7 MB/s 
     |████████████████████████████████| 170 kB 70.9 MB/s 
     |████████████████████████████████| 4.2 MB 80.1 MB/s 
     |████████████████████████████████| 75 kB 4.3 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 122 kB 78.6 MB/s 
     |████████████████████████████████| 786 kB 61.8 MB/s 
     |████████████████████████████████| 368 kB 74.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.19 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.0.3 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.26.0 which is incompatibl

In [3]:
%%writefile app.py
import cv2
from tensorflow.keras.models import load_model
import streamlit as st
from keras.preprocessing.image import load_img
import tensorflow as tf
import numpy as np
from tempfile import NamedTemporaryFile
from PIL import Image
import matplotlib.pyplot as plt


@st.cache(allow_output_mutation=True)
def load():
    return load_model('/content/drive/My Drive/pix2pix_model.h5')

def load_input(image_file1):
  input_image= tf.io.read_file(image_file1)
  input_image= tf.image.decode_jpeg(input_image)
  input_image = tf.cast(input_image, tf.float32)
  return input_image# real_image

def resize(input_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  return input_image

def normalize(input_image):
  input_image = (input_image / 127.5) - 1
  return input_image

def load_image_train(image_file1):
  img = tf.keras.preprocessing.image.array_to_img(image_file1)
  array = tf.keras.preprocessing.image.img_to_array(img)
  input_image = normalize(array)
  return input_image

def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  return prediction[0]

def main():

  st.write("""
  # Face2Comic with Pix2Pix  
  """)
  st.write("""
  # 내 얼굴을 미국 만화 캐릭터로~ 
  """)

  uploaded_file = st.file_uploader("Choose an image...", type="jpg")
  if uploaded_file is not None:
      image = Image.open(uploaded_file)
      st.image(image, caption='Uploaded Image.', use_column_width=True)
      st.write("")
      st.write("Converting...")
      input_image = load_image_train(image)
      input_image=resize(input_image,512,512)
      input_image=input_image[:,:,:3]
      model = load()
      prediction = generate_images(model,input_image[tf.newaxis, ...],input_image[tf.newaxis, ...])
      image = (np.array(prediction)+1)*127.5
      image = image.astype(np.uint8)
      image = Image.fromarray(image)
      st.image(image, caption='Predicted Image.', use_column_width=True)

if __name__ == '__main__':
	main()

Writing app.py


In [4]:
!ls

app.py	drive  sample_data


In [5]:
!ngrok authtoken 1wbrb36Sn8uyrAALbesmlvQy3lY_5QaBYPgTxN6ojrRyKe1TZ

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
from pyngrok import ngrok

In [7]:
!streamlit run app.py&>/dev/null&

In [8]:
publ_url = ngrok.connect('8501')

In [9]:
publ_url

<NgrokTunnel: "http://0560844b5146.ngrok.io" -> "http://localhost:8501">

In [10]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:01 node
     18 ?        00:00:00 tail
     64 ?        00:00:00 jupyter-noteboo
     65 ?        00:00:00 dap_multiplexer
     77 ?        00:00:02 python3
     99 ?        00:00:00 python3
    132 ?        00:00:00 bash
    133 ?        00:00:00 drive
    134 ?        00:00:00 grep
    252 ?        00:00:00 drive
    326 ?        00:00:00 bash
    327 ?        00:00:00 tail
    328 ?        00:00:00 python3
    383 ?        00:00:00 streamlit
    386 ?        00:00:00 ngrok
    400 ?        00:00:00 ps


In [11]:
# !kill 3657 3725 3771 
# ngrok.kill()

In [12]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:01 node
     18 ?        00:00:00 tail
     64 ?        00:00:00 jupyter-noteboo
     65 ?        00:00:00 dap_multiplexer
     77 ?        00:00:02 python3
     99 ?        00:00:00 python3
    132 ?        00:00:00 bash
    133 ?        00:00:00 drive
    134 ?        00:00:00 grep
    252 ?        00:00:00 drive
    326 ?        00:00:00 bash
    327 ?        00:00:00 tail
    328 ?        00:00:00 python3
    383 ?        00:00:00 streamlit
    386 ?        00:00:00 ngrok
    401 ?        00:00:00 ps
